In [2]:
!pip install spacy
!pip install spacy[transformers]
!python -m spacy download en_core_web_trf
!pip install datasets

Error in sys.excepthook:
Traceback (most recent call last):
  File "/usr/lib/python3.10/linecache.py", line 46, in getlines
    return updatecache(filename, module_globals)
  File "/usr/lib/python3.10/linecache.py", line 136, in updatecache
    with tokenize.open(fullname) as fp:
  File "/usr/lib/python3.10/tokenize.py", line 396, in open
    encoding, lines = detect_encoding(buffer.readline)
  File "/usr/lib/python3.10/tokenize.py", line 365, in detect_encoding
    first = read_or_stop()
  File "/usr/lib/python3.10/tokenize.py", line 323, in read_or_stop
    return readline()
KeyboardInterrupt

Original exception was:
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 187, in _run_module_as_main
    mod_name, mod_spec, code = _get_module_details(mod_name, _Error)
  File "/usr/lib/python3.10/runpy.py", line 146, in _get_module_details
    return _get_module_details(pkg_main_name, error)
  File "/usr/lib/python3.10/runpy.py", line 110, in _get_module_details


In [3]:
import spacy
from sklearn.model_selection import train_test_split
from spacy.training import Example
from spacy.tokens import DocBin
import re
from datasets import load_dataset
from spacy.scorer import Scorer
import random

In [4]:
# Loaded the base model for fine-tuning
nlp = spacy.load("en_core_web_trf")
stopwords = nlp.Defaults.stop_words

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`.

In [5]:
# function to convert dataset format for spaCy training
def prepare_data_for_spacy(conll_split):
    examples = []
    label_mapping = conll_split.features["ner_tags"].feature.int2str  
    for example in conll_split:
        text = " ".join(example['tokens'])
        entities = []
        start = 0
        for token, label in zip(example['tokens'], example['ner_tags']):
            end = start + len(token)
            if label != 0:  # 0 represents 'O' in CoNLL-2003 for non-entities
                # Converted numeric label to string using the label mapping
                label_str = label_mapping(label)
                entities.append((start, end, label_str))
            start = end + 1  
        examples.append((text, {"entities": entities}))
    return examples

In [6]:
conll_data = load_dataset("conll2003")

train_data = prepare_data_for_spacy(conll_data['train'])
val_data = prepare_data_for_spacy(conll_data['validation'])
test_data = prepare_data_for_spacy(conll_data['test'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [7]:
def convert_to_spacy_format(data, nlp):
    doc_bin = DocBin()
    for text, annot in data:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                ents.append(span)
        doc.ents = ents
        doc_bin.add(doc)
    return doc_bin

In [8]:
train_bin = convert_to_spacy_format(train_data, nlp)
val_bin = convert_to_spacy_format(val_data, nlp)
test_bin = convert_to_spacy_format(test_data, nlp)

# Save for training
train_bin.to_disk("./train.spacy")
val_bin.to_disk("./val.spacy")
test_bin.to_disk("./test.spacy")

In [9]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --gpu


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

✘ The provided output file already exists. To force overwriting the
config file, set the --force or -F flag.



In [13]:
!python -m spacy train config.cfg --output ./output --gpu-id 0


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWa

In [14]:
nlp_ner = spacy.load("./output/model-best")


def evaluate_model(nlp, data):
    scorer = Scorer()
    examples = []  

    for input_text, annotations in data:
        doc = nlp(input_text)  # Got predictions
        example = Example.from_dict(doc, annotations)  # Created Example object
        examples.append(example)  # Added the example to the list

    # Score the list of examples
    scores=scorer.score(examples)  

    return scores

test_scores = evaluate_model(nlp_ner, test_data)

In [15]:
if "ents_per_type" in test_scores:
    for entity, metrics in test_scores["ents_per_type"].items():
        print(f"Entity: {entity}")
        print(f"  Precision: {metrics['p']:.3f}")
        print(f"  Recall: {metrics['r']:.3f}")
        print(f"  F1-score: {metrics['f']:.3f}")
else:
    print("No entity type metrics found.")

Entity: B-LOC
  Precision: 0.954
  Recall: 0.915
  F1-score: 0.934
Entity: B-PER
  Precision: 0.969
  Recall: 0.970
  F1-score: 0.969
Entity: I-PER
  Precision: 0.979
  Recall: 0.984
  F1-score: 0.981
Entity: I-LOC
  Precision: 0.862
  Recall: 0.903
  F1-score: 0.882
Entity: B-MISC
  Precision: 0.825
  Recall: 0.845
  F1-score: 0.835
Entity: I-MISC
  Precision: 0.634
  Recall: 0.778
  F1-score: 0.699
Entity: B-ORG
  Precision: 0.881
  Recall: 0.934
  F1-score: 0.907
Entity: I-ORG
  Precision: 0.879
  Recall: 0.938
  F1-score: 0.907


In [20]:
def test_custom_text(nlp, custom_text):
    # Process the custom text with the model
    doc = nlp(custom_text)

    # Extract the named entities
    entities = [(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
    for entity in entities:
      print(entity)

custom_input = "Tashkent is the capital of Uzbekistan, and Bobur lives in this country and works for XalqBank"
custom_entities = test_custom_text(nlp_ner, custom_input)


('Tashkent', 0, 8, 'B-LOC')
('Uzbekistan', 27, 37, 'B-LOC')
('Bobur', 43, 48, 'B-PER')
('XalqBank', 85, 93, 'B-ORG')
